# Prompt Experiments

In [1]:
import os
import json
import re
import pandas as pd
import sys
sys.path.append('../../app')
from utils import eval_using_model
from dotenv import load_dotenv
from ipynb.fs.defs.helper_functions import get_response, extract_scores, append_row_to_dataframe, langchain_scores

/opt/app-root/src/prompt-pr/api-docs-generation/notebooks/evaluation/../../app/utils.py:2: DeprecationWarning: Deprecated import of TextGenerationParameters from module genai.text.generation. Please use `from genai.schema import TextGenerationParameters`.
  from genai.text.generation import TextGenerationParameters
/opt/app-root/src/prompt-pr/api-docs-generation/notebooks/evaluation/../../app/utils.py:3: DeprecationWarning: Deprecated import of TextTokenizationParameters from module genai.text.tokenization. Please use `from genai.schema import TextTokenizationParameters`.
  from genai.text.tokenization import (
/opt/app-root/src/prompt-pr/api-docs-generation/notebooks/evaluation/../../app/utils.py:3: DeprecationWarning: Deprecated import of TextTokenizationReturnOptions from module genai.text.tokenization. Please use `from genai.schema import TextTokenizationReturnOptions`.
  from genai.text.tokenization import (
/opt/app-root/src/prompt-pr/api-docs-generation/notebooks/evaluation/../.

In [2]:
# make sure you have a .env file in the root folder with genaikey and genaiapi
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)
openai_key = os.getenv("OPENAI_API_KEY", None)

In [3]:
llm = ChatOpenAI(model="gpt-4", temperature=0)

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
data = {
    'prompt': [],
    'response': [],
    'langchain_helpfulness': [],
    'langchain_correctness': [],
    'langchain_logical': [],
}

In [5]:
def get_response(instruction, model_id, file, functions, classes, documentation, imports, other, functions_code, functions_doc, classes_code, classes_doc):


    DATASET_PATH = "../../data/raw/chunked_data.json"

    with open(DATASET_PATH, "r", encoding="utf-8") as f:
        data = json.load(f)

    code = data[file]["code_chunks"]

    actual_doc = data[file]["markdown"]

    functions_text = code["functions"]
    classes_text = code["classes"]
    documentation_text = code["documentation"]
    imports_text = code["imports"]
    other_text = code["other"]
    functions_code_text = code["functions_code"]
    functions_doc_text = code["functions_docstrings"]
    classes_code_text = code["classes_code"]
    classes_doc_text = code["classes_docstrings"]


    prompt = generate_prompt(
        instruction,
        functions=functions,
        functions_text=functions_text,
        classes=classes,
        classes_text=classes_text,
        documentation=documentation,
        documentation_text=documentation_text,
        imports=imports,
        imports_text=imports_text,
        other=other,
        other_text=other_text,
        functions_code=functions_code,
        functions_code_text=functions_code_text,
        functions_doc=functions_doc,
        functions_doc_text=functions_doc_text,
        classes_code=classes_code,
        classes_code_text=classes_code_text,
        classes_doc=classes_doc,
        classes_doc_text=classes_doc_text,
    )

    if model_id == "OpenAI/gpt3.5":
        result = generate_text_using_OpenAI(prompt, openai_key)

    else:
        result = generate_text(model_id, prompt, decoding_method="sample", max_new_tokens=1024, temperature=0.7, top_k=50, top_p=0.50, genai_key=api_key)
    
    return prompt, result, actual_doc

In [6]:
df = pd.DataFrame(data)

In [7]:
def append_row_to_dataframe(df, prompt, generated_patch):

    evaluator = load_evaluator("criteria", llm=llm, criteria="helpfulness")
    eval_result = evaluator.evaluate_strings(prediction=generated_patch, input=prompt)
    print(eval_result)
    langchain_helpfulness = eval_result['score']
    
    evaluator = load_evaluator("labeled_criteria", llm=llm, criteria="correctness")
    eval_result = evaluator.evaluate_strings(prediction=generated_patch, input=prompt, reference=actual_doc)
    print(eval_result)
    langchain_correctness = eval_result['score']

    custom_criteria = {
    "logical": "Is the output complete? Does it capture all required fields"
                    }
    eval_chain = load_evaluator(
    EvaluatorType.CRITERIA,
    criteria=custom_criteria,
    llm=llm
    )
    eval_result = eval_chain.evaluate_strings(prediction=generated_patch, input=prompt)
    print(eval_result)
    langchain_logical = eval_result['score']

    new_row = {
        'prompt': prompt,
        'response': generated_patch,
        'langchain_helpfulness' : langchain_helpfulness,
        'langchain_correctness' : langchain_correctness,
        'langchain_logical' : langchain_logical
    }

    df = df.append(new_row, ignore_index=True)

    return df

### Prompt 1 

In [66]:
instruction = """
You are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

3. Error Handling: Describe possible error responses and their meanings.

Make sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. Avoid speculative information and prioritize accuracy and completeness.
"""

#### Exp 1

In [9]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'transparency', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=False, functions_doc=False, classes_code=True, classes_doc=False)

**Class Name:** LogInclusionProof

**1. Introduction:**
The LogInclusionProof class represents an inclusion proof for a log entry in a Merkle Tree. It provides methods to validate the proof and retrieve information about the proof.

**2. Properties:**

- `checkpoint` (StrictStr): The checkpoint associated with the inclusion proof.
- `hashes` (List[StrictStr]): The list of hashes in the inclusion proof path.
- `log_index` (StrictInt): The index of the log entry in the Merkle Tree.
- `root_hash` (StrictStr): The root hash of the Merkle Tree.
- `tree_size` (StrictInt): The current size of the Merkle Tree.

**3. Methods:**

3.1. `__init__(self, checkpoint: StrictStr, hashes: List[StrictStr], log_index: StrictInt, root_hash: StrictStr, tree_size: StrictInt)`
- Description: Initializes a new instance of the LogInclusionProof class.
- Parameters:
  - `checkpoint` (StrictStr): The checkpoint associated with the inclusion proof.
  - `hashes` (List[StrictStr]): The list of hashes in the inclusio

In [10]:
print(prompt)


You are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

3. Error Handling: Describe possible error responses and their meanings.

Make sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. Avoid speculative information and prioritize accuracy and completeness.


        
Class code:

class LogInclusionProof(BaseModel):
    

    model_config = ConfigDict(populate_by_name=True)

    checkpoint: StrictStr = Field(..., alias="che

In [11]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission is to be evaluated based on whether it is helpful, insightful, and appropriate.\n\nLooking at the submission, it provides a detailed documentation of the Python class "LogInclusionProof". It includes an introduction that explains the purpose of the class, a list of properties with their data types and descriptions, and a list of methods with their descriptions, parameters, and return values. This is helpful for anyone who needs to understand what this class does and how to use it.\n\nThe submission also includes a section on error handling, which describes the exceptions that the class may raise. This is insightful as it helps users understand the potential errors they might encounter when using this class and how to handle them.\n\nThe submission is appropriate as it follows the output structure provided in the task input. It is clear, concise, accurate, and user-centric, avoiding speculative information and p

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [12]:
df

,prompt,response,langchain_helpfulness,langchain_correctness,langchain_logical
0,\nYou are an AI system specialized at generati...,**Class Name:** LogInclusionProof\n\n**1. Intr...,1.0,0.0,0.0


#### Exp 2

In [67]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'oidc', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=False, classes_doc=False)

1. Introduction: 

The `detect_credential` function is used to detect the user's credential for a given audience in order to authenticate with an identity provider. It returns the detected credential or None if no credential is found.

2. Function: detect_credential()

    Description:
    
    The `detect_credential` function detects the user's credential for a given audience. It internally calls the `id.detect_credential` function with a default audience to detect the credential. If a credential is found, it is returned; otherwise, None is returned.
    
    Parameters:
    
    This function does not accept any parameters.
    
    Return Value:
    
    - Returns the detected credential as a string if found.
    - Returns None if no credential is found.
    
3. Error Handling:

    - If an error occurs during the detection of the credential, an IdentityError is raised. This can happen if there is an issue with the identity provider or if the credential cannot be detected for any re

In [68]:
print(prompt)


You are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

3. Error Handling: Describe possible error responses and their meanings.

Make sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. Avoid speculative information and prioritize accuracy and completeness.


Function Code:

def detect_credential() -> Optional[str]:
    
    try:
        return cast(Optional[str], id.detect_credential(_DEFAULT_AUDIENCE))
    except id.Ident

In [70]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate.\n\nLooking at the submission:\n\n1. Introduction: The submission provides a clear and concise introduction to the `detect_credential` function. It explains what the function does and its intended use, which is helpful for users trying to understand the function.\n\n2. Function: The submission provides a detailed description of the `detect_credential` function. It explains what the function does, the parameters it takes (or in this case, doesn\'t take), and the return value. This is insightful and helpful for users trying to understand how to use the function.\n\n3. Error Handling: The submission describes the possible error that can occur during the execution of the function and how it is handled. This is helpful for users trying to understand what might go wrong when using the function and how to handle such situations.\n\nOverall, the submission is helpful, insi

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


### Prompt 2

In [13]:
instruction = """
You are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Class: If a class code is passed, document the following:
    - Class Name and describe what it does.
    - Class Attributes - List and describe each attribute, including data types and any constraints.
    - Document each function in the class following the instructions below.
3. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

4. Error Handling: Describe each possible error response and the meaning of each error response.

Make sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. Avoid speculative information and prioritize accuracy and completeness.
Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documentation for the code that is actually present.
"""

#### Exp 1

In [14]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'transparency', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=False, functions_doc=False, classes_code=True, classes_doc=False)

**Class: LogInclusionProof**

The LogInclusionProof class represents an inclusion proof in a hash tree data structure. It is used to prove the inclusion of a specific data item in the hash tree.

**Class Attributes:**

1. `model_config` - A dictionary containing the model configuration. This attribute uses the ConfigDict class.
2. `checkpoint` - The checkpoint of the hash tree.
    - Data Type: StrictStr
    - Constraints: This attribute is required.
3. `hashes` - A list of hashes that form the inclusion proof.
    - Data Type: List[StrictStr]
    - Constraints: This attribute is required.
4. `log_index` - The index of the data item in the hash tree.
    - Data Type: StrictInt
    - Constraints: This attribute is required and must be a positive integer.
5. `root_hash` - The root hash of the hash tree.
    - Data Type: StrictStr
    - Constraints: This attribute is required.
6. `tree_size` - The size of the hash tree.
    - Data Type: StrictInt
    - Constraints: This attribute is requi

In [15]:
print(prompt)


You are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Class: If a class code is passed, document the following:
    - Class Name and describe what it does.
    - Class Attributes - List and describe each attribute, including data types and any constraints.
    - Document each function in the class following the instructions below.
3. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

4. Error Handling: Describe each possible error response and the meaning of each error response.

Make sure to follow this output structure to create API d

In [16]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate.\n\nLooking at the submission, it provides a detailed documentation for the given Python class code. It follows the structure provided in the input, including an introduction, class attributes, functions, and error handling.\n\nThe introduction gives a brief overview of the class and its purpose. The class attributes section lists all the attributes of the class, their data types, and constraints. The functions section provides a detailed explanation of each function, their parameters, and return values. The error handling section describes the possible errors that can occur and their meanings.\n\nThe submission is insightful as it provides a deep understanding of the class code. It explains the purpose of each attribute and function, and how they interact with each other. It also provides a clear explanation of the possible errors, which can be very helpful for us

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [17]:
df

,prompt,response,langchain_helpfulness,langchain_correctness,langchain_logical
0,\nYou are an AI system specialized at generati...,**Class Name:** LogInclusionProof\n\n**1. Intr...,1.0,0.0,0.0
1,\nYou are an AI system specialized at generati...,**Class: LogInclusionProof**\n\nThe LogInclusi...,1.0,1.0,1.0


#### Exp 2 

In [40]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'transparency', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=False, classes_doc=False)

# API Documentation

## Introduction
The API provides functions for manipulating and analyzing user data. It allows users to perform various operations such as calculating statistics, filtering data, and generating reports.

## Class: UserDataAnalyzer
This class provides methods for analyzing user data.

### Class Attributes
- `data` (list): A list of user data dictionaries. Each dictionary represents a user and has the following keys:
  - `'name'` (str): The name of the user.
  - `'age'` (int): The age of the user.
  - `'gender'` (str): The gender of the user.
  - `'income'` (float): The annual income of the user.

### Methods

#### `__init__(self, data: List[Dict[str, Union[str, int, float]]]) -> None`
Constructor method to initialize the UserDataAnalyzer object with the user data.

##### Parameters
- `data` (list): A list of dictionaries representing the user data. Each dictionary has the following keys:
    - `'name'` (str): The name of the user.
    - `'age'` (int): The age of the

In [41]:
print(prompt)


You are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Class: If a class code is passed, document the following:
    - Class Name and describe what it does.
    - Class Attributes - List and describe each attribute, including data types and any constraints.
    - Document each function in the class following the instructions below.
3. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

4. Error Handling: Describe each possible error response and the meaning of each error response.

Make sure to follow this output structure to create API d

### Prompt 3

#### Exp 1

In [46]:
instruction = """
You are an AI system specialized at generating API documentation for given Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Class: If a class code is passed, document the following:
    - Class Name and describe what it does.
    - Class Attributes - List and describe each attribute, including data types and any constraints.
    - Document each function in the class following the instructions below.
3. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

4. Error Handling: Describe each possible error response and the meaning of each error response.

Make sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. 

Special Caution:

- If no code is present in the prompt, do not generate documentation, simply state "No Code has been provided in the prompt".
- Avoid speculative information and prioritize accuracy and completeness.
- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documentation for the code that is actually present.
"""

In [47]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'transparency', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=False, classes_doc=False)

No code has been provided in the prompt.


In [48]:
print(prompt)


You are an AI system specialized at generating API documentation for given Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Class: If a class code is passed, document the following:
    - Class Name and describe what it does.
    - Class Attributes - List and describe each attribute, including data types and any constraints.
    - Document each function in the class following the instructions below.
3. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

4. Error Handling: Describe each possible error response and the meaning of each error response.

Make sure to follow this output structure to create API document

#### Exp 2

In [51]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'errors', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=False, classes_doc=False)

# **API Documentation**

## Introduction
This API is designed to provide functionality related to user authentication and authorization. It allows users to create accounts, log in, and perform actions that require authentication.

## Class

### UserAuth

The `UserAuth` class provides methods for user authentication.

#### Attributes

- `username` (str): The username of the user.
- `password` (str): The password of the user.

#### Methods

##### `__init__(self, username: str, password: str) -> None`

Constructs a new `UserAuth` object with the provided username and password.

###### Parameters
- `username` (str): The username of the user.
- `password` (str): The password of the user.

##### `login(self) -> bool`

Logs in the user with the provided username and password.

###### Returns
- `bool`: True if the login was successful, False otherwise.

##### `logout(self) -> None`

Logs out the user.

##### `change_password(self, new_password: str) -> None`

Changes the password of the user t

In [53]:
print(prompt)


You are an AI system specialized at generating API documentation for given Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Class: If a class code is passed, document the following:
    - Class Name and describe what it does.
    - Class Attributes - List and describe each attribute, including data types and any constraints.
    - Document each function in the class following the instructions below.
3. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

4. Error Handling: Describe each possible error response and the meaning of each error response.

Make sure to follow this output structure to create API document

In [55]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate.\n\nLooking at the submission, it is clear that it is helpful. It provides a detailed documentation of the API, including an introduction, class details, function details, and error handling. This would be very useful for a developer trying to understand how to use this API.\n\nThe submission is also insightful. It provides a clear explanation of what each function does, what parameters it takes, and what it returns. It also explains what each error means, which would be very useful for debugging.\n\nFinally, the submission is appropriate. It follows the structure outlined in the input, and it does not include any speculative information or hallucinated variable names, function names, class names, or intended API usage.\n\nTherefore, the submission meets the criterion of being helpful, insightful, and appropriate.', 'value': 'Therefore, the submission meets the cri

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


### Prompt 4

In [89]:
instruction = """
You are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. 

The documentation follow the structure below:

1. Introduction: 
2. Class: If a class code is passed, document the following:
    - Class Name and Description
    - Class Attributes and Data types
    - Document each function in the class following the instructions below.
3. Functions: 
    - Description
    - Parameters and Data types
    - Return Values

4. Error Handling: Possible error responses

Create API documentation that is clear, concise, accurate, and user-centric. 

Special Caution:

- If no code is present in the prompt, do not generate generic examples, simply state "No Code has been provided in the prompt".
- Avoid speculative information and prioritize accuracy and completeness.
- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documentation for the code that is actually present.
"""

#### Exp 1

In [58]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'transparency', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=False, classes_doc=False)

No code has been provided in the prompt.


#### Exp 2

In [60]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'errors', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=False, classes_doc=False)

No code has been provided in the prompt.


#### Exp 3

In [61]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'sign', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=False, classes_doc=False)

### Introduction:
This is the API documentation for a Python code.

### No Code has been provided in the prompt.


#### Exp 4

In [62]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'oidc', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=False, classes_doc=False)

1. Introduction:
This is a function called `detect_credential` that is used to detect credentials.

2. Function: `detect_credential`

    Description:
    This function is used to detect credentials. It attempts to detect the credentials by calling the `id.detect_credential()` function with a default audience. If an `id.IdentityError` is raised during the detection process, it is handled by raising another `IdentityError`.
    
    Parameters:
    This function does not take any parameters.
    
    Return Value:
    This function returns an optional string, which represents the detected credentials. If no credentials are detected, None is returned.
    
3. Error Handling:
   Possible error responses include an `IdentityError` being raised during the detection process. This error is handled by raising another `IdentityError`.


In [63]:
print(prompt)


You are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. 

The documentation follow the structure below:

1. Introduction: 
2. Class: If a class code is passed, document the following:
    - Class Name and Description
    - Class Attributes and Data types
    - Document each function in the class following the instructions below.
3. Functions: 
    - Description
    - Parameters and Data types
    - Return Values

4. Error Handling: Possible error responses

Create API documentation that is clear, concise, accurate, and user-centric. 

Special Caution:

- If no code is present in the prompt, do not generate generic examples, simply state "No Code has been provided in the prompt".
- Avoid speculative information and prioritize accuracy and completeness.
- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documenta

In [64]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate.\n\n1. Helpfulness: The submission provides a detailed explanation of the function `detect_credential`. It explains what the function does, the parameters it takes, the return value, and how it handles errors. This information is helpful for someone trying to understand the function.\n\n2. Insightfulness: The submission provides insights into how the function works. It explains that the function attempts to detect credentials by calling another function and handles any errors that occur during this process. This information provides insights into the inner workings of the function.\n\n3. Appropriateness: The submission is appropriate. It follows the structure provided in the prompt and provides all the necessary information. It does not include any speculative information or hallucinate any details.\n\nBased on this analysis, the submission meets the criterion of b

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


#### Exp 5

In [90]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'oidc', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=False, functions_doc=False, classes_code=True, classes_doc=False)

No code has been provided in the prompt.


In [91]:
print(prompt)


You are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. 

The documentation follow the structure below:

1. Introduction: 
2. Class: If a class code is passed, document the following:
    - Class Name and Description
    - Class Attributes and Data types
    - Document each function in the class following the instructions below.
3. Functions: 
    - Description
    - Parameters and Data types
    - Return Values

4. Error Handling: Possible error responses

Create API documentation that is clear, concise, accurate, and user-centric. 

Special Caution:

- If no code is present in the prompt, do not generate generic examples, simply state "No Code has been provided in the prompt".
- Avoid speculative information and prioritize accuracy and completeness.
- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documenta

In [92]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate. \n\nLooking at the data, the input provided a Python code for which the AI was supposed to generate API documentation. The code includes several classes and functions. \n\nHowever, the submission states "No code has been provided in the prompt." This is incorrect as there is clearly a Python code provided in the input. \n\nTherefore, the submission is not helpful or appropriate as it does not provide the required API documentation for the provided Python code. \n\nSo, the submission does not meet the criterion. \n\nN', 'value': 'N', 'score': 0}
{'reasoning': 'The criteria is to assess if the submission is correct, accurate, and factual. \n\nThe task was to generate API documentation for the provided Python code. The code provided was a Python script with several classes and functions. \n\nThe submission, however, states "No code has been provided in the prompt." T

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


#### Exp 6

In [93]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'sign', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=False, functions_doc=False, classes_code=True, classes_doc=False)

**Class Signer**

This class represents a signer that is responsible for signing artifacts. It takes an `identity_token`, `signing_ctx`, and an optional `cache` parameter. The `identity_token` is used to verify the identity of the signer, the `signing_ctx` provides the necessary context and dependencies for signing, and the `cache` parameter determines whether to cache the private key and signing certificate.

Attributes:
- `_identity_token`: The identity token used for verifying the signer's identity.
- `_signing_ctx`: The signing context that provides the necessary dependencies for signing.
- `__cached_private_key`: An optional cached private key.
- `__cached_signing_certificate`: An optional cached signing certificate.

Methods:
- `__init__(identity_token: IdentityToken, signing_ctx: SigningContext, cache: bool = True)`: Initializes the Signer instance with the provided `identity_token`, `signing_ctx`, and `cache` parameters. If `cache` is `True`, it generates an ephemeral private k

In [94]:
print(prompt)


You are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. 

The documentation follow the structure below:

1. Introduction: 
2. Class: If a class code is passed, document the following:
    - Class Name and Description
    - Class Attributes and Data types
    - Document each function in the class following the instructions below.
3. Functions: 
    - Description
    - Parameters and Data types
    - Return Values

4. Error Handling: Possible error responses

Create API documentation that is clear, concise, accurate, and user-centric. 

Special Caution:

- If no code is present in the prompt, do not generate generic examples, simply state "No Code has been provided in the prompt".
- Avoid speculative information and prioritize accuracy and completeness.
- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documenta

In [95]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission is to be evaluated based on whether it is helpful, insightful, and appropriate.\n\nLooking at the submission, it provides a detailed explanation of the classes and their methods. It describes the purpose of each class, their attributes, and the functionality of their methods. The submission also provides the data types of the parameters and return values, which is very helpful for understanding the code.\n\nThe submission is insightful as it not only describes what each method does but also explains the context in which they are used. For example, it explains that the `sign` method in the `Signer` class is used to sign an input artifact and that it verifies the validity period of the identity token, retrieves the signing certificate, verifies the SCT, and signs the artifact.\n\nThe submission is appropriate as it follows the structure provided in the prompt. It provides an introduction for each class, documents

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


#### Exp 7

In [96]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'transparency', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=False, functions_doc=False, classes_code=True, classes_doc=False)

**Class:** LogInclusionProof

- **Description**: This class represents an inclusion proof in a log. It contains various attributes and methods to validate the proof.

**Class Attributes:**

- **model_config**: A configuration dictionary for the model. (Type: ConfigDict)
- **checkpoint**: The checkpoint of the inclusion proof. (Type: StrictStr)
- **hashes**: A list of hashes involved in the proof. (Type: List[StrictStr])
- **log_index**: The index of the log in the proof. (Type: StrictInt)
- **root_hash**: The root hash of the log. (Type: StrictStr)
- **tree_size**: The size of the tree. (Type: StrictInt)

**Methods:**

- **_log_index_positive(v: int) -> int**: A field validator method that checks if the log index is a positive integer. Raises a ValueError if the log index is less than 0. (Parameters: v - the log index to validate) (Return Type: int)

- **_tree_size_positive(v: int) -> int**: A field validator method that checks if the tree size is a positive integer. Raises a ValueErro

In [97]:
print(prompt)


You are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. 

The documentation follow the structure below:

1. Introduction: 
2. Class: If a class code is passed, document the following:
    - Class Name and Description
    - Class Attributes and Data types
    - Document each function in the class following the instructions below.
3. Functions: 
    - Description
    - Parameters and Data types
    - Return Values

4. Error Handling: Possible error responses

Create API documentation that is clear, concise, accurate, and user-centric. 

Special Caution:

- If no code is present in the prompt, do not generate generic examples, simply state "No Code has been provided in the prompt".
- Avoid speculative information and prioritize accuracy and completeness.
- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documenta

In [98]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission is to be evaluated based on whether it is helpful, insightful, and appropriate.\n\nLooking at the submission, it provides a detailed documentation of the given Python class. It includes the class name and a brief description of what the class represents. This is helpful for users who want to understand the purpose of the class.\n\nThe submission also documents the class attributes, including their data types and what they represent. This is insightful as it provides users with a clear understanding of the data that the class handles.\n\nThe methods of the class are also well-documented. The submission provides the description, parameters, and return values for each method. This is appropriate as it gives users a clear understanding of how to use the methods and what to expect from them.\n\nThe submission also includes error handling, documenting the possible errors that can be raised when using the class. This 

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


#### Exp 8

In [99]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'oidc', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=True, classes_doc=False)

No code has been provided in the prompt.


In [100]:
print(prompt)


You are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. 

The documentation follow the structure below:

1. Introduction: 
2. Class: If a class code is passed, document the following:
    - Class Name and Description
    - Class Attributes and Data types
    - Document each function in the class following the instructions below.
3. Functions: 
    - Description
    - Parameters and Data types
    - Return Values

4. Error Handling: Possible error responses

Create API documentation that is clear, concise, accurate, and user-centric. 

Special Caution:

- If no code is present in the prompt, do not generate generic examples, simply state "No Code has been provided in the prompt".
- Avoid speculative information and prioritize accuracy and completeness.
- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documenta

In [101]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate. \n\nLooking at the data, the task was to generate API documentation for the provided Python code. The code includes a function and several classes. The task also specifies that if no code is present in the prompt, the assistant should state "No Code has been provided in the prompt".\n\nIn the submission, the assistant states "No code has been provided in the prompt". However, this is incorrect as there is clearly code provided in the input. Therefore, the assistant\'s response is not helpful or appropriate as it does not provide the requested API documentation for the provided code.\n\nN', 'value': 'N', 'score': 0}
{'reasoning': 'The task requires the AI to generate API documentation for the provided Python code. The code provided includes a function and several classes. The AI is supposed to generate documentation for these, following the structure provided in th

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


#### Exp 9

In [102]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'verify_models', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=True, functions_doc=False, classes_code=False, classes_doc=False)

**API Documentation**

1. Introduction:

No code has been provided in the prompt.

No code has been provided in the prompt.


2. Class:

No code has been provided in the prompt.


3. Functions:

No code has been provided in the prompt.


4. Error Handling:

No code has been provided in the prompt.


In [103]:
print(prompt)


You are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. 

The documentation follow the structure below:

1. Introduction: 
2. Class: If a class code is passed, document the following:
    - Class Name and Description
    - Class Attributes and Data types
    - Document each function in the class following the instructions below.
3. Functions: 
    - Description
    - Parameters and Data types
    - Return Values

4. Error Handling: Possible error responses

Create API documentation that is clear, concise, accurate, and user-centric. 

Special Caution:

- If no code is present in the prompt, do not generate generic examples, simply state "No Code has been provided in the prompt".
- Avoid speculative information and prioritize accuracy and completeness.
- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documenta

In [104]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission is supposed to provide API documentation for a given Python code. However, the prompt does not provide any code. The submission correctly identifies this and states "No code has been provided in the prompt" for each section of the documentation. This is accurate and appropriate given the lack of code in the prompt. Therefore, the submission can be considered helpful as it correctly identifies the lack of code and does not attempt to generate documentation for non-existent code. \n\nY', 'value': 'Y', 'score': 1}
{'reasoning': 'The criterion for this task is "correctness: Is the submission correct, accurate, and factual?"\n\nThe submission is asked to generate API documentation for a given Python code. However, no code was provided in the input. The submission correctly states "No code has been provided in the prompt" for each section of the documentation, which is accurate and factual. Therefore, the submission 

/tmp/ipykernel_1163/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Note:

The updated prompt is working well where we are not providing any code, that is it is able to detect clearly that no code has been given in thr prompt, but sometimes when a long chunk of code is present, it also says that "No code has been provided" probably becuase it fails to parse through big chunks,

rather than updating prompts, a better solution would be to chunk better where we just give it single classes and single functions to document.

### Prompt 5

In [13]:
instruction = """
Generate API documentation for Python code provided in the prompt. Ensure clarity, accuracy, and user-centricity.
If no code is provided, do not speculate or generate generic examples. Instead, leave this section blank or state "No code provided".

If Python code is provided:

1. Introduction: 
2. Class Documentation:
  - Document each class present in the code, including:
    - Class Name and Description
    - Class Attributes and Data types
    - Documentation for each method within the class, following the instructions below.
3. Function Documentation:
  - For each function in the code:
    - Function Description
    - Parameters, including names and data types.
    - Return values, including data types.
4. Error Handling:
Describe possible error responses and how they are handled in the code."""

#### Exp 1

In [20]:
prompt, generated_text, actual_doc = get_response(instruction, "OpenAI/gpt3.5", 'verify_models', functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=False, functions_doc=False, classes_code=True, classes_doc=False)

1. VerificationResult:
   - Description: This class represents the result of a verification process. It contains a boolean attribute "success" which indicates whether the verification was successful or not.
   - Attributes:
     - success (bool): Flag indicating the success of the verification.

   - Method Documentation:
     - __bool__(self) -> bool:
       - Description: This method allows the VerificationResult object to be used as a boolean value. It returns the value of the "success" attribute.
       - Returns: bool - The value of the "success" attribute.

2. VerificationSuccess:
   - Description: This class represents a successful verification result. It is a subclass of VerificationResult and inherits its attributes and methods.
   - Attributes:
     - success (bool): Flag indicating the success of the verification. It is set to True.

3. VerificationFailure:
   - Description: This class represents a failed verification result. It is a subclass of VerificationResult and inheri

In [21]:
print(prompt)


Generate API documentation for Python code provided in the prompt. Ensure clarity, accuracy, and user-centricity.
If no code is provided, do not speculate or generate generic examples. Instead, leave this section blank or state "No code provided".

If Python code is provided:

1. Introduction: 
2. Class Documentation:
  - Document each class present in the code, including:
    - Class Name and Description
    - Class Attributes and Data types
    - Documentation for each method within the class, following the instructions below.
3. Function Documentation:
  - For each function in the code:
    - Function Description
    - Parameters, including names and data types.
    - Return values, including data types.
4. Error Handling:
Describe possible error responses and how they are handled in the code.
Class code:
class VerificationResult(BaseModel):
    

    success: bool
    

    def __bool__(self) -> bool:
        
        return self.success
class VerificationSuccess(VerificationResul

In [22]:
df = append_row_to_dataframe(df, prompt, generated_text)

{'reasoning': 'The criterion for this task is "helpfulness". The submission is to be evaluated based on whether it is helpful, insightful, and appropriate.\n\nLooking at the submission, it is clear that the user has provided detailed documentation for each class present in the Python code. The documentation includes the class name, a description of the class, the class attributes and their data types, and documentation for each method within the class. This is in line with the instructions provided in the input.\n\nThe documentation is also user-centric, as it provides clear and concise descriptions of each class and method, making it easy for users to understand the purpose and functionality of each component of the code.\n\nThe submission is also accurate, as it correctly identifies and describes each class and method in the code. It correctly identifies the data types of the class attributes and return values of the methods.\n\nThe submission does not include a section on error hand

/tmp/ipykernel_4392/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


## Run automated experiment

For all models and chunks generate outputs and score them and see which prompt has higher average score

### Experiment 1

In [8]:
instruction_1 = """
You are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. 

The documentation follow the structure below:

1. Introduction: 
2. Class: If a class code is passed, document the following:
    - Class Name and Description
    - Class Attributes and Data types
    - Document each function in the class following the instructions below.
3. Functions: 
    - Description
    - Parameters and Data types
    - Return Values

4. Error Handling: Possible error responses

Create API documentation that is clear, concise, accurate, and user-centric. 

Special Caution:

- If no code is present in the prompt, do not generate generic examples, simply state "No Code has been provided in the prompt".
- Avoid speculative information and prioritize accuracy and completeness.
- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documentation for the code that is actually present.
"""

In [9]:
instruction_2 = """
Generate API documentation for Python code provided in the prompt. Ensure clarity, accuracy, and user-centricity.
If no code is provided, do not speculate or generate generic examples. Instead, leave this section blank or state "No code provided".

If Python code is provided:

1. Introduction: 
2. Class Documentation:
  - Document each class present in the code, including:
    - Class Name and Description
    - Class Attributes and Data types
    - Documentation for each method within the class, following the instructions below.
3. Function Documentation:
  - For each function in the code:
    - Function Description
    - Parameters, including names and data types.
    - Return values, including data types.
4. Error Handling:
Describe possible error responses and how they are handled in the code."""

In [10]:
instruction_old = """
You are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:

1. Introduction: Briefly describe the purpose of the API and its intended use.
2. Functions: Document each API function, including:
    - Description: Clearly explain what the endpoint or function does.
    - Parameters: List and describe each parameter, including data types and any constraints.
    - Return Values: Specify the data type and possible values returned.

3. Error Handling: Describe possible error responses and their meanings.

Make sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. Avoid speculative information and prioritize accuracy and completeness.
"""

In [11]:
results_df = pd.DataFrame(columns=['model', 'prompt', 'code_file', 'part', 'response', 'langchain_helpfulness', 'langchain_correctness', 'langchain_logical'])

models = ["OpenAI/gpt3.5"]
instruction_options = [instruction_1, instruction_2, instruction_old]
code_files = ["oidc", "transparency", "errors", "verify_models", "verify_policy", "sign", "verify_verifier"]
enabled_parts = ["functions_code", "classes_code"]

for model in models:
    for inst in instruction_options:
        for code_file in code_files:
            for part in enabled_parts:
                prompt, generated_text, actual_doc = get_response(inst, model, code_file, functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=(part=="functions_code"), functions_doc=False, classes_code=(part=="classes_code"), classes_doc=False)
                results_df = append_row_to_dataframe(results_df, prompt, generated_text)

                other_values = {'model': model,
                                'code_file': code_file,
                                'part': part,
                                'instruction': inst
                               }
                for column, value in other_values.items():
                    results_df.loc[results_df.index[-1], column] = value

results_df['total_langchain_score'] = results_df['langchain_helpfulness'] + results_df['langchain_correctness'] + results_df['langchain_logical']

1. Introduction:
This API function is used to detect a credential. It returns an optional string value representing the detected credential.

2. Function:
- Description:
    - This function attempts to detect a credential.
- Parameters and Data Types:
    - No parameters are required.
- Return Value:
    - This function returns an optional string value representing the detected credential. If no credential is detected, it returns None.

3. Error Handling:
- Possible error responses:
    - If an error occurs during the detection process, an `IdentityError` is raised.
{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate.\n\nLooking at the submission:\n\n1. The introduction provides a brief overview of the function\'s purpose, which is helpful for users to understand what the function does.\n\n2. The function description is clear and concise, providing insight into the function\'s operation. It correctly states that the

/tmp/ipykernel_224/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


No Code has been provided in the prompt.
{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate. \n\nLooking at the data, the input provided a Python code and asked for API documentation to be created for it. The code includes several classes and functions that need to be documented. \n\nHowever, the submission states "No Code has been provided in the prompt." This is incorrect as there is clearly a Python code provided in the input. \n\nTherefore, the submission is not helpful or appropriate as it does not provide the requested API documentation for the provided Python code. It is also not insightful as it does not provide any useful information or analysis. \n\nBased on this reasoning, the submission does not meet the criterion.\n\nN', 'value': 'N', 'score': 0}
{'reasoning': 'The criteria is to assess if the submission is correct, accurate, and factual. \n\nThe submission states "No Code has been provided in the prom

/tmp/ipykernel_224/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


**1. Introduction:**

The Python script provided includes classes and functions that perform specific tasks. This documentation will guide you through the usage and functionality of the API.

**2. Class:**


2.1 MyClass

   This class represents a sample class and provides some useful functions to perform certain tasks.

   **Attributes:**

   - `attribute1` (int): Represents the first attribute of MyClass.
   - `attribute2` (str): Represents the second attribute of MyClass.

   **Methods:**

   - `__init__(self, attribute1: int, attribute2: str) -> None`: Initializes the MyClass object with the provided attribute values.

   - `method1(self, param1: str, param2: int) -> str`: Represents the first method of MyClass. It takes in two parameters, `param1` of type str and `param2` of type int, and returns a string.

   - `method2(self) -> None`: Represents the second method of MyClass. It doesn't take any parameters and doesn't return anything.


**3. Functions:**


3.1 `function1(param1: 

...

/tmp/ipykernel_224/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


# API Documentation

## Introduction
This API provides functions to perform operations on a list of numbers.

## Functions

### `add_numbers(numbers: List[int]) -> int`
This function takes a list of integers as input and returns the sum of all the numbers in the list.

#### Parameters
- `numbers` (List[int]): A list of integers.

#### Return Value
- Returns an integer that represents the sum of all the numbers in the list.

### `multiply_numbers(numbers: List[int]) -> int`
This function takes a list of integers as input and returns the product of all the numbers in the list.

#### Parameters
- `numbers` (List[int]): A list of integers.

#### Return Value
- Returns an integer that represents the product of all the numbers in the list.

## Error Handling
- If the input list is empty, both functions will return 0, as there are no numbers to sum or multiply.
- If the input list contains non-integer values, both functions will raise a `TypeError` with a message indicating that the input sho

/tmp/ipykernel_224/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


**Introduction:**

The Verifier class is part of a larger API that provides functionality for verifying the authenticity and integrity of artifacts. The Verifier class specifically handles the verification process by performing various checks and validations on the provided signing materials and Rekor entries.

**Functions:**

1. `__init__(
        self, *, rekor: RekorClient, fulcio_certificate_chain: List[Certificate]
    )`
    
    - Description: Initializes an instance of the Verifier class with the provided RekorClient instance and list of Fulcio certificates.
    - Parameters:
        - `rekor` (RekorClient): An instance of the RekorClient class, used for fetching Rekor entries.
        - `fulcio_certificate_chain` (List[Certificate]): A list of Fulcio certificates used for verifying the signing certificate.
    - Return Values: None

2. `production(cls) -> Verifier`
    
    - Description: Returns a production instance of the Verifier class with the necessary configurations.
  

/tmp/ipykernel_224/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [12]:
results_df

,model,prompt,code_file,part,response,langchain_helpfulness,langchain_correctness,langchain_logical,instruction,total_langchain_score
0,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,oidc,functions_code,1. Introduction:\nThis API function is used to...,1,1,1,\nYou are an AI system specialized at generati...,3
1,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,oidc,classes_code,No Code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0
2,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,transparency,functions_code,**1. Introduction:**\n\nThe Python script prov...,1,0,1,\nYou are an AI system specialized at generati...,2
3,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,transparency,classes_code,**Class Name:** LogInclusionProof\n\n**Descrip...,None,1,1,\nYou are an AI system specialized at generati...,NaN
4,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,errors,functions_code,# Introduction:\nNo Code has been provided in ...,0,1,1,\nYou are an AI system specialized at generati...,2
5,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,errors,classes_code,No code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0
6,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_models,functions_code,No code has been provided in the prompt.,1,1,1,\nYou are an AI system specialized at generati...,3
7,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_models,classes_code,No code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0
8,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_policy,functions_code,# **API Documentation**\n\n## Introduction:\nT...,0,None,1,\nYou are an AI system specialized at generati...,NaN
9,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_policy,classes_code,No code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0


In [13]:
grouped = results_df.groupby(['model', 'code_file', 'part'])

filtered_groups = grouped.filter(lambda x: not x[['langchain_helpfulness', 'langchain_correctness', 'langchain_logical']].isnull().any().any())

In [15]:
valid_groups = filtered_groups.groupby(['model', 'code_file', 'part']).filter(lambda x: set(x['instruction']) == set(results_df['instruction']))

In [28]:
valid_groups.to_csv("results_1.csv")

In [24]:
total_scores = valid_groups.groupby('instruction')[['langchain_helpfulness', 'langchain_correctness', 'langchain_logical']].sum()
row_counts = valid_groups.groupby('instruction').size()

print("Total Scores per Instruction:")
total_scores

Total Scores per Instruction:


,langchain_helpfulness,langchain_correctness,langchain_logical
instruction,,,
"\nGenerate API documentation for Python code provided in the prompt. Ensure clarity, accuracy, and user-centricity.\nIf no code is provided, do not speculate or generate generic examples. Instead, leave this section blank or state ""No code provided"".\n\nIf Python code is provided:\n\n1. Introduction: \n2. Class Documentation:\n - Document each class present in the code, including:\n - Class Name and Description\n - Class Attributes and Data types\n - Documentation for each method within the class, following the instructions below.\n3. Function Documentation:\n - For each function in the code:\n - Function Description\n - Parameters, including names and data types.\n - Return values, including data types.\n4. Error Handling:\nDescribe possible error responses and how they are handled in the code.",8,6,7
"\nYou are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. \n\nThe documentation follow the structure below:\n\n1. Introduction: \n2. Class: If a class code is passed, document the following:\n - Class Name and Description\n - Class Attributes and Data types\n - Document each function in the class following the instructions below.\n3. Functions: \n - Description\n - Parameters and Data types\n - Return Values\n\n4. Error Handling: Possible error responses\n\nCreate API documentation that is clear, concise, accurate, and user-centric. \n\nSpecial Caution:\n\n- If no code is present in the prompt, do not generate generic examples, simply state ""No Code has been provided in the prompt"".\n- Avoid speculative information and prioritize accuracy and completeness.\n- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documentation for the code that is actually present.\n",4,3,5
"\nYou are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:\n\n1. Introduction: Briefly describe the purpose of the API and its intended use.\n2. Functions: Document each API function, including:\n - Description: Clearly explain what the endpoint or function does.\n - Parameters: List and describe each parameter, including data types and any constraints.\n - Return Values: Specify the data type and possible values returned.\n\n3. Error Handling: Describe possible error responses and their meanings.\n\nMake sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. Avoid speculative information and prioritize accuracy and completeness.\n",8,8,7


In [25]:
human_scored_valid_groups = pd.read_csv("human_scored_results_1.csv")

In [26]:
total_scores = human_scored_valid_groups.groupby('instruction')[['langchain_helpfulness', 'langchain_correctness', 'langchain_logical']].sum()
row_counts = human_scored_valid_groups.groupby('instruction').size()

print("Total Scores per Instruction:")
total_scores

Total Scores per Instruction:


,langchain_helpfulness,langchain_correctness,langchain_logical
instruction,,,
"Generate API documentation for Python code provided in the prompt. Ensure clarity, accuracy, and user-centricity.\n If no code is provided, do not speculate or generate generic examples. Instead, leave this section blank or state ""No code provided"".\n \n\n If Python code is provided:\n \n\n 1. Introduction: \n 2. Class Documentation:\n - Document each class present in the code, including:\n - Class Name and Description\n - Class Attributes and Data types\n - Documentation for each method within the class, following the instructions below.\n 3. Function Documentation:\n - For each function in the code:\n - Function Description\n - Parameters, including names and data types.\n - Return values, including data types.\n 4. Error Handling:\n Describe possible error responses and how they are handled in the code.",8,8,7
"You are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. \n \n\n The documentation follow the structure below:\n \n\n 1. Introduction: \n 2. Class: If a class code is passed, document the following:\n - Class Name and Description\n - Class Attributes and Data types\n - Document each function in the class following the instructions below.\n 3. Functions: \n - Description\n - Parameters and Data types\n - Return Values\n \n\n 4. Error Handling: Possible error responses\n \n\n Create API documentation that is clear, concise, accurate, and user-centric. \n \n\n Special Caution:\n \n\n - If no code is present in the prompt, do not generate generic examples, simply state ""No Code has been provided in the prompt"".\n - Avoid speculative information and prioritize accuracy and completeness.\n - Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documentation for the code that is actually present.",2,2,2
"You are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:\n \n\n 1. Introduction: Briefly describe the purpose of the API and its intended use.\n 2. Functions: Document each API function, including:\n - Description: Clearly explain what the endpoint or function does.\n - Parameters: List and describe each parameter, including data types and any constraints.\n - Return Values: Specify the data type and possible values returned.\n \n\n 3. Error Handling: Describe possible error responses and their meanings.\n \n\n Make sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. Avoid speculative information and prioritize accuracy and completeness.",3,3,2


In [27]:
print("\nNumber of Rows Accounted for per Instruction:")
row_counts


Number of Rows Accounted for per Instruction:


instruction
Generate API documentation for Python code provided in the prompt. Ensure clarity, accuracy, and user-centricity.\n If no code is provided, do not speculate or generate generic examples. Instead, leave this section blank or state "No code provided".\n \n\n If Python code is provided:\n \n\n 1. Introduction: \n 2. Class Documentation:\n  - Document each class present in the code, including:\n  - Class Name and Description\n  - Class Attributes and Data types\n  - Documentation for each method within the class, following the instructions below.\n 3. Function Documentation:\n  - For each function in the code:\n  - Function Description\n  - Parameters, including names and data types.\n  - Return values, including data types.\n 4. Error Handling:\n Describe possible error responses and how they are handled in the code.                                                                                                                                                                   

### Experiment 2

In [11]:
results_df = pd.DataFrame(columns=['model', 'prompt', 'code_file', 'part', 'response', 'langchain_helpfulness', 'langchain_correctness', 'langchain_logical'])

models = ["OpenAI/gpt3.5", "ibm/granite-20b-code-instruct-v1"]
instruction_options = [instruction_1, instruction_2, instruction_old]
code_files = ["oidc", "transparency", "errors", "verify_models", "verify_policy", "sign", "verify_verifier"]
enabled_parts = ["functions_code", "classes_code"]

for model in models:
    for inst in instruction_options:
        for code_file in code_files:
            for part in enabled_parts:
                prompt, generated_text, actual_doc = get_response(inst, model, code_file, functions=False, classes=False, documentation=False, imports=False, other=False, functions_code=(part=="functions_code"), functions_doc=False, classes_code=(part=="classes_code"), classes_doc=False)
                results_df = append_row_to_dataframe(results_df, prompt, generated_text)

                other_values = {'model': model,
                                'code_file': code_file,
                                'part': part,
                                'instruction': inst
                               }
                for column, value in other_values.items():
                    results_df.loc[results_df.index[-1], column] = value

results_df['total_langchain_score'] = results_df['langchain_helpfulness'] + results_df['langchain_correctness'] + results_df['langchain_logical']

1. Introduction:
The detect_credential function is used to detect the credential for a given audience.

2. Functions:

- **detect_credential()**:
    - Description: This function detects the credential for a given audience.
    - Parameters: There are no parameters for this function.
    - Return Value: It returns an optional string, which is the detected credential for the specified audience.

3. Error Handling:
- If an IdentityError exception is raised during the credential detection process, it will be caught and raised again as an IdentityError exception.
{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate.\n\n1. Helpfulness: The submission provides a clear and concise explanation of the function, its parameters, return value, and error handling. It explains that the function is used to detect the credential for a given audience, and that it returns an optional string. It also explains that the function does not

/tmp/ipykernel_1030/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


No Code has been provided in the prompt.
{'reasoning': 'The criterion for this task is "helpfulness". The submission should be helpful, insightful, and appropriate. \n\nLooking at the data, the input provided a Python code and asked for API documentation. The submission, however, states "No Code has been provided in the prompt." This is incorrect as there is clearly a Python code provided in the input. \n\nTherefore, the submission is not helpful or appropriate as it does not provide the requested API documentation for the given Python code. It is also not insightful as it does not provide any useful information or analysis. \n\nSo, the submission does not meet the criterion.\n\nN', 'value': 'N', 'score': 0}
{'reasoning': 'The criteria is to assess the correctness of the submission. The submission is supposed to generate API documentation for the provided Python code. \n\nThe provided Python code includes several classes and functions such as "_OpenIDConfiguration", "ExpiredIdentity", 

/tmp/ipykernel_1030/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


# **API Documentation**

## Introduction:
Welcome to the API documentation for the Python code.

## Functions:
### Description:
#### Example Function Name
This function takes in two parameters `param1` and `param2` and returns the sum of the two parameters.

### Parameters and Data Types:
- param1: int - The first parameter.
- param2: int - The second parameter.

### Return Values:
- int: The sum of `param1` and `param2`.

## Error Handling:
Possible error responses:
- If either `param1` or `param2` is not an integer, a `TypeError` will be raised.
{'reasoning': 'The criterion for this task is "helpfulness". The submission is supposed to be helpful, insightful, and appropriate.\n\n1. Helpfulness: The submission provides a clear and concise description of a hypothetical function, its parameters, return values, and possible error responses. This would be helpful to a user trying to understand how to use the function.\n\n2. Insightfulness: The submission provides insight into the function\

...

/tmp/ipykernel_1030/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


generated_text="\n## Example\n\n```\nIntroduction: This API provides access to a database of user information.\n\nFunctions:\n\nget_user(user_id):\nDescription: Retrieve information about a specific user.\nParameters:\n    user_id (int): The unique identifier for the user.\nReturn Values:\n    user (dict): A dictionary containing information about the user.\n\ncreate_user(name, email):\nDescription: Create a new user account.\nParameters:\n    name (str): The user's name.\n    email (str): The user's email address.\nReturn Values:\n    user (dict): A dictionary containing information about the new user.\n\nError Handling:\n\nInvalidUserError: The specified user does not exist.\n```\n\n## Submission\n\nPlease submit your API documentation as a text file or a PDF file." generated_token_count=180 generated_tokens=None input_text=None input_token_count=165 input_tokens=None moderation=None seed=3535149533.0 stop_reason='eos_token' stop_sequence=None
{'reasoning': 'The criterion for this ta

/tmp/ipykernel_1030/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


generated_text='\nIntroduction:\n\nThe Verifier class is responsible for verifying the integrity and authenticity of a software artifact. It takes a VerificationMaterials object and a VerificationPolicy object as input, and returns a VerificationResult object.\n\nFunctions:\n\nverify:\n\nThis is the main function of the Verifier class. It takes a VerificationMaterials object and a VerificationPolicy object as input, and returns a VerificationResult object.\n\nParameters:\n\nmaterials (VerificationMaterials): The VerificationMaterials object containing the artifact to be verified, its signature, and its signing certificate.\n\npolicy (VerificationPolicy): The VerificationPolicy object containing the rules for verifying the artifact.\n\nReturn Values:\n\nVerificationResult: The VerificationResult object containing the result of the verification process, which can be either a VerificationSuccess object or a VerificationFailure object.\n\nError Handling:\n\nIf the signing certificate is no

/tmp/ipykernel_1030/1423965122.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [12]:
results_df

,model,prompt,code_file,part,response,langchain_helpfulness,langchain_correctness,langchain_logical,instruction,total_langchain_score
0,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,oidc,functions_code,1. Introduction:\nThe detect_credential functi...,1,1,0,\nYou are an AI system specialized at generati...,2
1,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,oidc,classes_code,No Code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0
2,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,transparency,functions_code,# **API Documentation**\n\n## Introduction:\nW...,0,0,0,\nYou are an AI system specialized at generati...,0
3,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,transparency,classes_code,No code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0
4,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,errors,functions_code,# API Documentation\n\n## Introduction:\n\nWel...,1,1,1,\nYou are an AI system specialized at generati...,3
...,...,...,...,...,...,...,...,...,...,...
79,ibm/granite-20b-code-instruct-v1,\nYou are an AI system specialized at generati...,verify_policy,classes_code,Introduction:\nThis module contains classes fo...,0,0,0,\nYou are an AI system specialized at generati...,0
80,ibm/granite-20b-code-instruct-v1,\nYou are an AI system specialized at generati...,sign,functions_code,\n## Example\n\n```\nIntroduction:\nThe API pr...,1,0,1,\nYou are an AI system specialized at generati...,2
81,ibm/granite-20b-code-instruct-v1,\nYou are an AI system specialized at generati...,sign,classes_code,Introduction:\nThis API is designed to be used...,1,0,0,\nYou are an AI system specialized at generati...,1
82,ibm/granite-20b-code-instruct-v1,\nYou are an AI system specialized at generati...,verify_verifier,functions_code,\n## Example\n\n```\nIntroduction: This API pr...,1,1,1,\nYou are an AI system specialized at generati...,3


In [14]:
grouped = results_df.groupby(['model', 'code_file', 'part'])

filtered_groups = grouped.filter(lambda x: not x[['langchain_helpfulness', 'langchain_correctness', 'langchain_logical']].isnull().any().any())

In [15]:
valid_groups = filtered_groups.groupby(['model', 'code_file', 'part']).filter(lambda x: set(x['instruction']) == set(results_df['instruction']))

In [16]:
valid_groups.to_csv("results_2.csv")

In [17]:
total_scores = valid_groups.groupby('instruction')[['langchain_helpfulness', 'langchain_correctness', 'langchain_logical']].sum()
row_counts = valid_groups.groupby('instruction').size()

print("Total Scores per Instruction:")
total_scores

Total Scores per Instruction:


,langchain_helpfulness,langchain_correctness,langchain_logical
instruction,,,
"\nGenerate API documentation for Python code provided in the prompt. Ensure clarity, accuracy, and user-centricity.\nIf no code is provided, do not speculate or generate generic examples. Instead, leave this section blank or state ""No code provided"".\n\nIf Python code is provided:\n\n1. Introduction: \n2. Class Documentation:\n - Document each class present in the code, including:\n - Class Name and Description\n - Class Attributes and Data types\n - Documentation for each method within the class, following the instructions below.\n3. Function Documentation:\n - For each function in the code:\n - Function Description\n - Parameters, including names and data types.\n - Return values, including data types.\n4. Error Handling:\nDescribe possible error responses and how they are handled in the code.",17,10,9
"\nYou are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. \n\nThe documentation follow the structure below:\n\n1. Introduction: \n2. Class: If a class code is passed, document the following:\n - Class Name and Description\n - Class Attributes and Data types\n - Document each function in the class following the instructions below.\n3. Functions: \n - Description\n - Parameters and Data types\n - Return Values\n\n4. Error Handling: Possible error responses\n\nCreate API documentation that is clear, concise, accurate, and user-centric. \n\nSpecial Caution:\n\n- If no code is present in the prompt, do not generate generic examples, simply state ""No Code has been provided in the prompt"".\n- Avoid speculative information and prioritize accuracy and completeness.\n- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documentation for the code that is actually present.\n",8,8,5
"\nYou are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:\n\n1. Introduction: Briefly describe the purpose of the API and its intended use.\n2. Functions: Document each API function, including:\n - Description: Clearly explain what the endpoint or function does.\n - Parameters: List and describe each parameter, including data types and any constraints.\n - Return Values: Specify the data type and possible values returned.\n\n3. Error Handling: Describe possible error responses and their meanings.\n\nMake sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. Avoid speculative information and prioritize accuracy and completeness.\n",20,14,12


In [19]:
human_scored_valid_groups = pd.read_csv("human_scored_results_2.csv")

In [20]:
total_scores = human_scored_valid_groups.groupby('instruction')[['langchain_helpfulness', 'langchain_correctness', 'langchain_logical']].sum()
row_counts = human_scored_valid_groups.groupby('instruction').size()

print("Total Scores per Instruction:")
total_scores

Total Scores per Instruction:


,langchain_helpfulness,langchain_correctness,langchain_logical
instruction,,,
"\nGenerate API documentation for Python code provided in the prompt. Ensure clarity, accuracy, and user-centricity.\nIf no code is provided, do not speculate or generate generic examples. Instead, leave this section blank or state ""No code provided"".\n\nIf Python code is provided:\n\n1. Introduction: \n2. Class Documentation:\n - Document each class present in the code, including:\n - Class Name and Description\n - Class Attributes and Data types\n - Documentation for each method within the class, following the instructions below.\n3. Function Documentation:\n - For each function in the code:\n - Function Description\n - Parameters, including names and data types.\n - Return values, including data types.\n4. Error Handling:\nDescribe possible error responses and how they are handled in the code.",14,13,9
"\nYou are an AI system specialized at generating API documentation for given Python code. Be as objective as possible. You will be provided functions, classes, or Python scripts. \n\nThe documentation follow the structure below:\n\n1. Introduction: \n2. Class: If a class code is passed, document the following:\n - Class Name and Description\n - Class Attributes and Data types\n - Document each function in the class following the instructions below.\n3. Functions: \n - Description\n - Parameters and Data types\n - Return Values\n\n4. Error Handling: Possible error responses\n\nCreate API documentation that is clear, concise, accurate, and user-centric. \n\nSpecial Caution:\n\n- If no code is present in the prompt, do not generate generic examples, simply state ""No Code has been provided in the prompt"".\n- Avoid speculative information and prioritize accuracy and completeness.\n- Do not hallucinate variable names, function names, class names and the intended API usage. Only generate documentation for the code that is actually present.\n",8,8,6
"\nYou are an AI system specialized at generating API documentation for the provided Python code. You will be provided functions, classes, or Python scripts. Your documentation should include:\n\n1. Introduction: Briefly describe the purpose of the API and its intended use.\n2. Functions: Document each API function, including:\n - Description: Clearly explain what the endpoint or function does.\n - Parameters: List and describe each parameter, including data types and any constraints.\n - Return Values: Specify the data type and possible values returned.\n\n3. Error Handling: Describe possible error responses and their meanings.\n\nMake sure to follow this output structure to create API documentation that is clear, concise, accurate, and user-centric. Avoid speculative information and prioritize accuracy and completeness.\n",10,7,3


In [21]:
print("\nNumber of Rows Accounted for per Instruction:")
row_counts


Number of Rows Accounted for per Instruction:


instruction
\nGenerate API documentation for Python code provided in the prompt. Ensure clarity, accuracy, and user-centricity.\nIf no code is provided, do not speculate or generate generic examples. Instead, leave this section blank or state "No code provided".\n\nIf Python code is provided:\n\n1. Introduction: \n2. Class Documentation:\n  - Document each class present in the code, including:\n    - Class Name and Description\n    - Class Attributes and Data types\n    - Documentation for each method within the class, following the instructions below.\n3. Function Documentation:\n  - For each function in the code:\n    - Function Description\n    - Parameters, including names and data types.\n    - Return values, including data types.\n4. Error Handling:\nDescribe possible error responses and how they are handled in the code.                                                                                                                                                                  

## Research Questions

### Q1. For the best prompt, on the basis of human evaluated scores, what are the scenarios where it is missing out or getting 0 scores and why?
        1. Which model?
        2. Is it because it creates hallucinated content?
        3. Is it because it cannot parse long code chunks?
    
### Q2. Where are the human evaluations and the langchain evaluations diverging?
       1. For which score, helpfulness, lofical or correctness do the langchain and human evaluations diverge the most.
       2. Where does the langchain eval give false positives?
       3. Where is it doing good?
       4. Are there more false positives or false negatives?

In [27]:
filtered_df = human_scored_valid_groups[human_scored_valid_groups['instruction'] == instruction_2]

In [28]:
# Group by model and count the number of 0s in each langchain category
langchain_zeros = filtered_df.groupby("model")[
    ["langchain_helpfulness", "langchain_correctness", "langchain_logical"]
].apply(lambda x: (x == 0).sum())

print("Number of 0s in each langchain category per model:")
langchain_zeros

Number of 0s in each langchain category per model:


,langchain_helpfulness,langchain_correctness,langchain_logical
model,,,
OpenAI/gpt3.5,1,1,2
ibm/granite-20b-code-instruct-v1,8,9,12


OpenAI has way less 0s than granite. So on the human evaluation, openAI is doing much better.

### Answer to Q1.1 The OpenAI model is generating more correct, helpful and logical outputs.

In [31]:
filtered_rows = filtered_df[
    (human_scored_valid_groups[["langchain_helpfulness", "langchain_correctness", "langchain_logical"]] == 0).sum(axis=1) >= 2
][["model", "part", "code_file", "response"]]

print("\nRows with 2 or 3 zeros in langchain scores:")
filtered_rows


Rows with 2 or 3 zeros in langchain scores:


/tmp/ipykernel_1030/2143233954.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = filtered_df[


model            part        code_file  \
11                     OpenAI/gpt3.5  functions_code             oidc   
47  ibm/granite-20b-code-instruct-v1  functions_code     transparency   
48  ibm/granite-20b-code-instruct-v1    classes_code           errors   
49  ibm/granite-20b-code-instruct-v1  functions_code    verify_models   
50  ibm/granite-20b-code-instruct-v1    classes_code    verify_models   
51  ibm/granite-20b-code-instruct-v1  functions_code    verify_policy   
52  ibm/granite-20b-code-instruct-v1    classes_code    verify_policy   
53  ibm/granite-20b-code-instruct-v1  functions_code             sign   
54  ibm/granite-20b-code-instruct-v1    classes_code             sign   
55  ibm/granite-20b-code-instruct-v1  functions_code  verify_verifier   
56  ibm/granite-20b-code-instruct-v1    classes_code  verify_verifier   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

 #### Model : Open AI
row 11 : OIDC has function code yet it said no code provided, it did not hallucinate, but did not do what it was asked to do
#### Model: Granite
row 47, 49, 51, 53, : It has no functions_code, it hallucinated content

row 48, 50, 52, 54, 56 : It generated API doc, but it is partially incorrect,

### Answer to Q1.2 and Q1.3 The granite model when it fails, it fails becuase half of the time it hallucinates content and half of the time it is not able to parse code accurately

In [37]:
pd.set_option('display.max_colwidth', 50)

In [34]:
human_scored_valid_groups.reset_index(drop=True, inplace=True)
valid_groups.reset_index(drop=True, inplace=True)

In [47]:
valid_groups.head(10)

,model,prompt,code_file,part,response,langchain_helpfulness,langchain_correctness,langchain_logical,instruction,total_langchain_score
0,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,oidc,functions_code,1. Introduction:\nThe detect_credential functi...,1,1,0,\nYou are an AI system specialized at generati...,2
1,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,oidc,classes_code,No Code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0
2,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,transparency,functions_code,# **API Documentation**\n\n## Introduction:\nW...,0,0,0,\nYou are an AI system specialized at generati...,0
3,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,errors,functions_code,# API Documentation\n\n## Introduction:\n\nWel...,1,1,1,\nYou are an AI system specialized at generati...,3
4,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_models,functions_code,No code has been provided in the prompt.,1,1,1,\nYou are an AI system specialized at generati...,3
5,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_models,classes_code,No Code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0
6,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_policy,functions_code,No code has been provided in the prompt.,1,1,1,\nYou are an AI system specialized at generati...,3
7,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,sign,functions_code,"```\nclass Dog:\n def __init__(self, name, ...",1,1,0,\nYou are an AI system specialized at generati...,2
8,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,sign,classes_code,**Class Signer**\n\nClass for signing artifact...,1,1,1,\nYou are an AI system specialized at generati...,3
9,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_verifier,functions_code,## API Documentation\n\n### Introduction:\nExa...,0,0,0,\nYou are an AI system specialized at generati...,0


In [48]:
human_scored_valid_groups.head(10)

,Unnamed: 0,model,prompt,code_file,part,response,langchain_helpfulness,langchain_correctness,langchain_logical,instruction,total_langchain_score
0,0,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,oidc,functions_code,1. Introduction:\nThe detect_credential functi...,1,1,0,\nYou are an AI system specialized at generati...,2
1,1,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,oidc,classes_code,No Code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0
2,2,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,transparency,functions_code,# **API Documentation**\n\n## Introduction:\nW...,0,0,0,\nYou are an AI system specialized at generati...,0
3,4,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,errors,functions_code,# API Documentation\n\n## Introduction:\n\nWel...,1,1,1,\nYou are an AI system specialized at generati...,3
4,6,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_models,functions_code,No code has been provided in the prompt.,1,1,1,\nYou are an AI system specialized at generati...,3
5,7,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_models,classes_code,No Code has been provided in the prompt.,0,0,0,\nYou are an AI system specialized at generati...,0
6,8,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_policy,functions_code,No code has been provided in the prompt.,1,1,1,\nYou are an AI system specialized at generati...,3
7,10,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,sign,functions_code,"```\nclass Dog:\n def __init__(self, name, ...",0,0,0,\nYou are an AI system specialized at generati...,0
8,11,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,sign,classes_code,**Class Signer**\n\nClass for signing artifact...,1,1,1,\nYou are an AI system specialized at generati...,3
9,12,OpenAI/gpt3.5,\nYou are an AI system specialized at generati...,verify_verifier,functions_code,## API Documentation\n\n### Introduction:\nExa...,1,1,1,\nYou are an AI system specialized at generati...,3


In [41]:
# Step 1: Find divergences and similarities
divergences = {}
similarities = {}

for score in ['langchain_helpfulness', 'langchain_correctness', 'langchain_logical']:
    divergences[score] = (human_scored_valid_groups[score] != valid_groups[score]).sum()
    similarities[score] = (human_scored_valid_groups[score] == valid_groups[score]).sum()

# Identify the score with the most and least divergence
most_divergent_score = max(divergences, key=divergences.get)
least_divergent_score = min(divergences, key=divergences.get)

print("Most divergent score:", most_divergent_score)
print("Number of rows:", len(human_scored_valid_groups))
print("Number of scores matching:", similarities[most_divergent_score])
print("Number of scores different:", divergences[most_divergent_score])
print()

print("Least divergent score:", least_divergent_score)
print("Number of rows:", len(human_scored_valid_groups))
print("Number of scores matching:", similarities[least_divergent_score])
print("Number of scores different:", divergences[least_divergent_score])

Most divergent score: langchain_helpfulness
Number of rows: 69
Number of scores matching: 54
Number of scores different: 15

Least divergent score: langchain_logical
Number of rows: 69
Number of scores matching: 59
Number of scores different: 10


### Answer to Q2.1 the langchain helpfulness score diverges more than the langchain logical score. Meaning the langchain criteria is better at scoring on correctness and logic, which is more important for us anyway

In [46]:
print("Helpfulness")
false_positives = valid_groups[(valid_groups['langchain_helpfulness'] == 1) & (human_scored_valid_groups['langchain_helpfulness'] == 0)]

false_positives_count = len(false_positives)
false_negatives_count = (valid_groups['langchain_helpfulness'] == 0).sum() - ((valid_groups['langchain_helpfulness'] == 0) & (human_scored_valid_groups['langchain_helpfulness'] == 1)).sum()

print("False positives count: (Automated langchain score is 1 but human score is 0)", false_positives_count)
print("False negatives count: (Automated langchain score is 0 but human score is 1)", false_negatives_count)


print("Correctness")
false_positives = valid_groups[(valid_groups['langchain_correctness'] == 1) & (human_scored_valid_groups['langchain_correctness'] == 0)]

false_positives_count = len(false_positives)
false_negatives_count = (valid_groups['langchain_correctness'] == 0).sum() - ((valid_groups['langchain_correctness'] == 0) & (human_scored_valid_groups['langchain_correctness'] == 1)).sum()

print("False positives count: (Automated langchain score is 1 but human score is 0)", false_positives_count)
print("False negatives count: (Automated langchain score is 0 but human score is 1)", false_negatives_count)


print("Logic")
false_positives = valid_groups[(valid_groups['langchain_logical'] == 1) & (human_scored_valid_groups['langchain_logical'] == 0)]

false_positives_count = len(false_positives)
false_negatives_count = (valid_groups['langchain_logical'] == 0).sum() - ((valid_groups['langchain_logical'] == 0) & (human_scored_valid_groups['langchain_logical'] == 1)).sum()

print("False positives count: (Automated langchain score is 1 but human score is 0)", false_positives_count)
print("False negatives count: (Automated langchain score is 0 but human score is 1)", false_negatives_count)

Helpfulness
False positives count: (Automated langchain score is 1 but human score is 0) 14
False negatives count: (Automated langchain score is 0 but human score is 1) 23
Correctness
False positives count: (Automated langchain score is 1 but human score is 0) 8
False negatives count: (Automated langchain score is 0 but human score is 1) 33
Logic
False positives count: (Automated langchain score is 1 but human score is 0) 9
False negatives count: (Automated langchain score is 0 but human score is 1) 42


### Answer to Q2.2 and Q2.3 There are way more false negatives than false positives. That means langchain is more conservative in scoring the results, it will give a 0 more easily than assigning a 1  

# Conclusion

1. The final prompt is doing better than the other prompts. 
2. The OpenAI model is generating more correct, helpful and logical outputs.
3. When he granite model it fails, it fails becuase half of the time it hallucinates content and half of the time it is not able to parse code accurately
4. Langchain is better at scoring on correctness and logic, than helpfulness.
5. There are way more false negatives than false positives. That means langchain is more conservative in scoring the results, it will give a 0 more easily than assigning a 1. 